# Object and scene detection using Amazon Rekognition

***
This notebook provides a walkthrough of [object detection API](https://docs.aws.amazon.com/rekognition/latest/dg/labels.html) in Amazon Rekognition to identify objects.
***

# Initialize stuff

In [ ]:
# Initialise Notebook
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont, ExifTags, ImageColor
import time
import os
import io

In [ ]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = "us-east-1"

In [ ]:
# Init clients
rekognition = boto3.client('rekognition', region_name='us-east-1')
s3 = boto3.client('s3')

In [ ]:
# S3 bucket that contains sample images and videos

# We are providing sample images and videos in this bucket so
# you do not have to manually download/upload test images and videos.

bucketName = "aws-rek-immersionday-" + awsRegion

In [ ]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and draw bounding boxes

!mkdir m1tmp
tempFolder = 'm1tmp/'

# Detect objects in image
***

## Detect Image using S3 Object

In [ ]:
imageName = "media/object-detection/cars.png"

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

#### Call Rekognition to detect objects in the image

In [ ]:
# Call Amazon Rekognition to detect objects in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectLabels.html

detectLabelsResponse = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Labels API (Object Detection)
# In the JSON response below, you will see Label, detected instances, confidence score and additional information.

display(detectLabelsResponse)

### Use AWS Rekognition using local file and bring your own image

- Upload your image to the static folder in this directory


In [ ]:
# Todo-1 upload your own image to static folder and assign imagePath with your own image
imagePath = ""
with open(imagePath, 'rb') as image:
    response = rekognition.detect_labels(Image={'Bytes': image.read()})

In [ ]:
img = Image.open(imagePath)
display(img)

In [ ]:
#Display rekognition JSON Response
display(response)

# Recognize objects in video
 Object recognition in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html. 

- First we start a label detection job which returns a Job Id.
- We can then call `get_label_detection` to get the job status and after job is complete, we can get object metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
***

In [ ]:
# Todo-2 download object-detection.mov file and upload to your s3, assign bucketName with your own bucket and videoName with the object key
bucketName=""
videoName=""
rekognition = boto3.client('rekognition')

#### Call Rekognition to start a job for object detection

In [ ]:
# Start video label recognition job
startLabelDetection = rekognition.start_label_detection(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

labelsJobId = startLabelDetection['JobId']
display("Job Id: {0}".format(labelsJobId))

### Additional (Optional) Request Attributes

ClientRequestTokenL
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html#rekognition-StartLabelDetection-request-ClientRequestToken

JobTag:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html#rekognition-StartLabelDetection-request-JobTag

MinConfidence:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html#rekognition-StartLabelDetection-request-MinConfidence

NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartLabelDetection.html#rekognition-StartLabelDetection-request-NotificationChannel


#### Wait for object detection job to complete

In [ ]:
# Wait for object detection job to complete
# In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
getObjectDetection = rekognition.get_label_detection(
    JobId=labelsJobId,
    SortBy='TIMESTAMP'
)

while(getObjectDetection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getObjectDetection = rekognition.get_label_detection(
    JobId=labelsJobId,
    SortBy='TIMESTAMP')
    
display(getObjectDetection['JobStatus'])

#### Review raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Object Detection API
# In the JSON response below, you will see list of detected objects and activities.
# For each detected object, you will see information like Timestamp

display(getObjectDetection)

#### Display names of recognized objects in the video

In [ ]:
flaggedObjectsInVideo = ["Person"]

theObjects = {}

# Display timestamps and objects detected at that time
strDetail = "Objects detected in video<br>=======================================<br>"
strOverall = "Objects in the overall video:<br>=======================================<br>"

# Objects detected in each frame
for obj in getObjectDetection['Labels']:
    ts = obj ["Timestamp"]
    cconfidence = obj['Label']["Confidence"]
    oname = obj['Label']["Name"]
    
    if(oname in flaggedObjectsInVideo):
        print("Found flagged object at {} ms: {} (Confidence: {})".format(ts, oname, round(cconfidence,2)))
    
    strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence,2))
    if oname in theObjects:
        cojb = theObjects[oname]
        theObjects[oname] = {"Name" : oname, "Count": 1+cojb["Count"]}
    else:
        theObjects[oname] = {"Name" : oname, "Count": 1}

# Unique objects detected in video
for theObject in theObjects:
    strOverall = strOverall + "Name: {}, Count: {}<br>".format(theObject, theObjects[theObject]["Count"])

# Display results
display(HTML(strOverall))

#### Show video in the player

In [ ]:
# Show video in a player

s3VideoUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': videoName})

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))

In [ ]:
listui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(strDetail)
display(HTML(listui))